# Get songs from Nike Run Club user
- Exploring songs uploaded by the spotify user Nike Run Club https://open.spotify.com/user/nikerunclub?si=30e01984c90c47d4.
- Use Spotify API https://developer.spotify.com/

## Get libraries

In [24]:
#!pip install spotify
#!pip install base64
#!pip install lyricsgenius

import spotify.sync as spotify
import base64
import datetime
import requests
from urllib.parse import urlencode
import pandas as pd
import os
current_path = os.getcwd()

## Set up API access

In [25]:
# Setting up spotify credentials
spotify_client_id = 'YOUR-CLIENT-ID'
spotify_client_secret = 'YOUR-SECRET-KEY'
spotify_client_creds = f'{spotify_client_id}:{spotify_client_secret}'
spotify_client_creds_b64 = base64.b64encode(spotify_client_creds.encode())
    
# Using client ID and secret to get a token to be used for each requests
# https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow
spotify_token_url = 'https://accounts.spotify.com/api/token' 
spotify_token_data = {
    'grant_type': 'client_credentials'
}

spotify_token_headers = {
    'Authorization': f"Basic {spotify_client_creds_b64.decode()}" #<base64 encoded client_id:client_secret>
}

# Get the token with a POST request
r = requests.post(spotify_token_url, data=spotify_token_data, headers=spotify_token_headers)
valid_request = r.status_code in range(200, 299)
if valid_request:
#     now = datetime.datetime.now
    spotify_access_token = r.json()['access_token']
#     expires_in = r.json()['expires_in'] # in seconds
#     expires = now + datetime.timedelta(seconds=expires_in)
#     did_expire = expires < now

spotify_headers = {
    "Authorization": f"Bearer {spotify_access_token}"
}

## Get 20 playlist uploaded by Nike Run Club

In [26]:
# API call from https://developer.spotify.com/documentation/web-api/reference/#category-playlists
user_id = "nikerunclub"
lookup_url = f"https://api.spotify.com/v1/users/{user_id}/playlists"

#spotify_playlist_id = spotify_single_playlist.json()["playlists"]["items"][0]["id"]
#endpoint = f'https://api.spotify.com/v1/playlists/{spotify_playlist_id}/tracks'
#data = urlencode({"limit":30, "offset":0})
#lookup_url = f"{endpoint}?{data}"

# Default limit of of 20 playlists
nike_run_club_playlists = requests.get(lookup_url, headers=spotify_headers)

# Define dictionary and database for playlist
playlists_dict = {'playlist_id': None, 'playlist_name': None, 'url': None, 'total_track': None}

columns_playlists = ['playlist_id', 'playlist_name', 'url', 'total_track']

df_playlists = pd.DataFrame(columns=columns_playlists)

In [27]:
nike_run_club_playlists.json()

{'href': 'https://api.spotify.com/v1/users/nikerunclub/playlists?offset=0&limit=20',
 'items': [{'collaborative': False,
   'description': 'Elite heptathlete Erica Bougard invites you into her world by sharing her favorite jams. Run, dance, or simply vibe out to the music that touches her soul.',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/4R0ssQxlJCZfhBecvXg7xb'},
   'href': 'https://api.spotify.com/v1/playlists/4R0ssQxlJCZfhBecvXg7xb',
   'id': '4R0ssQxlJCZfhBecvXg7xb',
   'images': [{'height': None,
     'url': 'https://i.scdn.co/image/ab67706c0000bebbd6654360bd9e02111447d8ed',
     'width': None}],
   'name': 'BETRUE with Erica',
   'owner': {'display_name': 'Nike Run Club',
    'external_urls': {'spotify': 'https://open.spotify.com/user/nikerunclub'},
    'href': 'https://api.spotify.com/v1/users/nikerunclub',
    'id': 'nikerunclub',
    'type': 'user',
    'uri': 'spotify:user:nikerunclub'},
   'primary_color': None,
   'public': True,
   'snapshot_id': 'M

In [28]:
# Store the results in a dataframe
for playlist in nike_run_club_playlists.json()['items']:
    playlists_dict['playlist_id'] = playlist['id']
    playlists_dict['playlist_name'] = playlist['name']
    playlists_dict['url'] = playlist['external_urls']['spotify']
    playlists_dict['total_track'] = playlist['tracks']['total']
    
    # Store in the pandas dataset
    df_playlists = df_playlists.append(playlists_dict, ignore_index=True)

df_playlists

,playlist_id,playlist_name,url,total_track
0,4R0ssQxlJCZfhBecvXg7xb,BETRUE with Erica,https://open.spotify.com/playlist/4R0ssQxlJCZf...,15
1,1H5JvGH5vahRJKC06OFENH,Nike Run Club,https://open.spotify.com/playlist/1H5JvGH5vahR...,18
2,0UFQxGiMJtm0v71eQrgyTX,Long Run,https://open.spotify.com/playlist/0UFQxGiMJtm0...,17
3,4RpqiyPxEHFdpdDuRkgrS5,Speed Run,https://open.spotify.com/playlist/4RpqiyPxEHFd...,17
4,7wsAeCVW2zEYDpJOlmuAZp,Run For Life,https://open.spotify.com/playlist/7wsAeCVW2zEY...,14
5,2p7jfepFtOz9B2mo6fzhAF,Mindful Miles,https://open.spotify.com/playlist/2p7jfepFtOz9...,15
6,6e78FeS1ZOSkiAxTxuQLkV,Slow Down 2 Speed Up,https://open.spotify.com/playlist/6e78FeS1ZOSk...,14
7,5UeYUUQnTy2amoVm6Tbmeb,Run with Headspace,https://open.spotify.com/playlist/5UeYUUQnTy2a...,15
8,7tnXSTcfVmztjQShPqmgEB,Run With Shalane,https://open.spotify.com/playlist/7tnXSTcfVmzt...,18
9,3gJX84BBycAJ11ddL7TNp4,Run with Eliud,https://open.spotify.com/playlist/3gJX84BBycAJ...,13


## Get songs in the selected Nike Run Club playlists

In [29]:
# Define a dict to keep information about songs
songs_dict = {'spotify_id': None, 'artist': None, 'artist_id': None, 'feat_artists': None, 'spotify_url': None, 'title': None, 'popularity': None,
              'lyrics': None, 'duration_ms': None, 'song_art_image_url': None,
             'danceability': None, 'energy': None, 'key': None, 'loudness': None, 'mode': None, 'speechiness': None, 
              'acousticness': None, 'instrumentalness': None, 'liveness': None, 'valence': None, 'tempo': None
}

# Define empti pandas dataframe to store song info
songs_col = ['spotify_id', 'title', 'artist', 'artist_id', 'feat_artists', 'spotify_url', 'popularity', 
                 'lyrics', 'duration_ms', 'song_art_image_url', 'danceability', 'energy', 
                 'key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 
                 'valence', 'tempo'
]

audio_features_col = ['popularity', 
                 'duration_ms', 'danceability', 'energy', 
                 'key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 
                 'valence', 'tempo']

metadata_col = ['spotify_id', 'title', 'artist', 'artist_id', 'feat_artists', 'spotify_url', 'song_art_image_url', 'lyrics']

df_songs = pd.DataFrame(columns=songs_col)

In [30]:
# Function definitions
def remove_brackets(mystring):
    ''' Remove brackets from the title, otherwise can not find lyrics from genius
    '''
    start = mystring.find("(")
    end = mystring.find(")")
    result = mystring
    
    if start != -1 and end != -1:
        result = mystring[0:start]
    return result

In [31]:
# Store data of songs in a dataframe
for spotify_id in df_playlists.playlist_id:
    # API call for each different playlists
    endpoint = f'https://api.spotify.com/v1/playlists/{spotify_id}/tracks'
    spotify_tracks_playlist = requests.get(endpoint, headers=spotify_headers)
    
    # Look at the different parameters
    #data = urlencode({"market": "IT", "limit":25, "offset":0, "fields": "items(track(name, album(artists, release_date), popularity, external_urls(spotify), duration_ms))"})
    #lookup_url = f"{endpoint}?{data}"
    
    # Loop inside each playlist to get data
    for iteration, song in enumerate(spotify_tracks_playlist.json()['items']):

        # Get information fom the playlist api call
        songs_dict['spotify_id'] = song['track']['id']
        songs_dict['title'] = remove_brackets(song['track']['name'])
        songs_dict['popularity'] = song['track']['popularity']
        songs_dict['spotify_url'] = song['track']['external_urls']['spotify']
        songs_dict['duration_ms'] = song['track']['duration_ms']

        # Get audio features information
        # more info here https://developer.spotify.com/documentation/web-api/reference/#object-audiofeaturesobject
        spotify_id = song['track']['id']
        endpoint = f"https://api.spotify.com/v1/audio-features/{spotify_id}"
        audio_features = requests.get(endpoint, headers=spotify_headers)
        songs_dict['danceability'] = audio_features.json()['danceability']
        songs_dict['energy'] = audio_features.json()['energy']
        songs_dict['key'] = audio_features.json()['key']
        songs_dict['loudness'] = audio_features.json()['loudness']
        songs_dict['mode'] = audio_features.json()['mode']
        songs_dict['spechiness'] = audio_features.json()['speechiness']
        songs_dict['acousticness'] = audio_features.json()['acousticness']
        songs_dict['instrumentalness'] = audio_features.json()['instrumentalness']
        songs_dict['liveness'] = audio_features.json()['liveness']
        songs_dict['valence'] = audio_features.json()['valence']
        songs_dict['tempo'] = audio_features.json()['tempo']

        # Look if there are multiple artists
        list_feat_artists = []

        for i, artist in enumerate(song['track']['album']['artists']):
            if i == 0:
                songs_dict['artist'] = artist['name']
                songs_dict['artist_id'] = artist['id']
            else:
                list_feat_artists.append(artist['name'])

        songs_dict['feat_artists'] = list_feat_artists  

        df_songs = df_songs.append(songs_dict, ignore_index=True)
    
df_songs.sort_values(by='popularity', ascending=False).head(10)

,spotify_id,title,artist,artist_id,feat_artists,spotify_url,popularity,lyrics,duration_ms,song_art_image_url,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,spechiness
30,0vR2rIVORmgeKiGIgNT0fV,By Your Side,Calvin Harris,7CajNmpbOovFoOoasH2HaY,[Tom Grennan],https://open.spotify.com/track/0vR2rIVORmgeKiG...,86,None,189671,None,...,7,-3.597,1,None,0.0514,0.001990,0.2870,0.811,124.010,0.0295
160,0azC730Exh71aQlOt9Zj3y,This Is What You Came For,Calvin Harris,7CajNmpbOovFoOoasH2HaY,[],https://open.spotify.com/track/0azC730Exh71aQl...,81,None,222160,None,...,9,-2.787,0,None,0.1990,0.119000,0.1480,0.465,123.962,0.0332
158,0lYBSQXN6rCTvUZvg9S0lU,Let Me Love You,DJ Snake,540vIaP2JwjQb9dm3aArA4,[],https://open.spotify.com/track/0lYBSQXN6rCTvUZ...,81,None,205946,None,...,8,-5.371,1,None,0.0863,0.000026,0.1350,0.163,99.988,0.0349
45,7B3z0ySL9Rr0XvZEAjWZzM,Sofia,Clairo,3l0CmX0FuQjFxr8SK7Vqag,[],https://open.spotify.com/track/7B3z0ySL9Rr0XvZ...,81,None,188386,None,...,0,-9.805,1,None,0.5980,0.003720,0.2310,0.641,112.997,0.0390
61,7HXBG0W8gFJwHUh5mVF9tf,Rise,Lost Frequencies,7f5Zgnp2spUuuzKplmRkt7,[],https://open.spotify.com/track/7HXBG0W8gFJwHUh...,80,None,193018,None,...,6,-6.014,1,None,0.2280,0.000008,0.3680,0.461,122.076,0.0387
6,7AzlLxHn24DxjgQX73F9fU,No Idea,Don Toliver,4Gso3d4CscCijv0lmajZWs,[],https://open.spotify.com/track/7AzlLxHn24DxjgQ...,80,None,154424,None,...,6,-5.718,0,None,0.5240,0.000579,0.1650,0.350,127.998,0.0893
25,0DmAvNCAK08oCi7miSZUIY,Lasting Lover,Sigala,1IueXOQyABrMOprrzwQJWN,[James Arthur],https://open.spotify.com/track/0DmAvNCAK08oCi7...,80,None,218358,None,...,1,-4.529,1,None,0.1970,0.000000,0.0943,0.483,125.983,0.0478
161,1i1fxkWeaMmKEB4T7zqbzK,Don't Let Me Down,The Chainsmokers,69GGBxA162lTqCwzJG5jLp,[],https://open.spotify.com/track/1i1fxkWeaMmKEB4...,79,None,208373,None,...,11,-5.094,1,None,0.1570,0.005080,0.1360,0.422,159.803,0.1720
178,3ebXMykcMXOcLeJ9xZ17XH,Scared to Be Lonely,Martin Garrix,60d24wfXkVzDSfLS6hyCjZ,[Dua Lipa],https://open.spotify.com/track/3ebXMykcMXOcLeJ...,78,None,220883,None,...,1,-7.786,0,None,0.0895,0.000000,0.2610,0.195,137.972,0.0576
9,6vN77lE9LK6HP2DewaN6HZ,Yes Indeed,Lil Baby,5f7VJjfbwm532GiveGC0ZK,[],https://open.spotify.com/track/6vN77lE9LK6HP2D...,78,None,142273,None,...,5,-9.309,0,None,0.0355,0.000000,0.1080,0.562,119.957,0.5300


It is worth mentioning that audio features describe:
- **popularity**: the popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past;
- **acousticness**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic;
- **danceability**: describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable;
- **duration_ms**: the duration of the track in milliseconds;
- **energy**: is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy;
- **instrumentalness**: predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0;
- **key**: The key the track is in. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on;
- **liveness**: detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live;
- **loudness**: the overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db;
- **mode**: mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0;
- **speechiness**: detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks;
- **tempo**: the overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration;
- **valence**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

In [20]:
# Look at what is considered as audio features
audio_features_col

['popularity',
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo']

In [16]:
# Look at only audio fearures
df_songs[audio_features_col]

,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,65,173610,0.782,0.500,5,-6.473,0,None,0.342000,0.000019,0.1330,0.2800,102.008
1,32,213933,0.616,0.516,7,-7.703,1,None,0.358000,0.000000,0.0941,0.2370,76.950
2,26,156960,0.943,0.670,4,-5.511,0,None,0.241000,0.000009,0.0974,0.8930,120.077
3,47,220487,0.745,0.446,2,-7.518,0,None,0.619000,0.000000,0.1130,0.0396,122.982
4,75,301893,0.704,0.487,9,-6.407,0,None,0.669000,0.000086,0.1110,0.3130,121.726
...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,30,126362,0.901,0.644,0,-11.638,1,None,0.000099,0.466000,0.1140,0.5450,111.046
267,19,71186,0.805,0.841,6,-7.610,1,None,0.403000,0.179000,0.6420,0.4420,91.001
268,24,46880,0.281,0.310,4,-14.788,1,None,0.165000,0.974000,0.6430,0.2750,65.761
269,32,171094,0.605,0.992,1,-5.074,1,None,0.006200,0.907000,0.4540,0.5560,102.894


In [22]:
# Look at what is considered metadata
metadata_col

['spotify_id',
 'title',
 'artist',
 'feat_artists',
 'spotify_url',
 'song_art_image_url',
 'lyrics']

In [23]:
df_songs[metadata_col]

,spotify_id,title,artist,feat_artists,spotify_url,song_art_image_url,lyrics
0,3mRlFZHUyvJbPTlkzg4LyJ,Roses,SAINt JHN,[],https://open.spotify.com/track/3mRlFZHUyvJbPTl...,None,None
1,2wB0rnmsJOv1eqrRjQB5yZ,My Affection,Summer Walker,[],https://open.spotify.com/track/2wB0rnmsJOv1eqr...,None,None
2,76DHyqVuAlzxQJZHFajoBY,Mood Swings,A Boogie Wit da Hoodie,[],https://open.spotify.com/track/76DHyqVuAlzxQJZ...,None,None
3,75VJBp3JE7W3oKoFeq1esH,Chicago Freestyle,Drake,[],https://open.spotify.com/track/75VJBp3JE7W3oKo...,None,None
4,51rPRW8NjxZoWPPjnRGzHw,Tadow,Masego,[],https://open.spotify.com/track/51rPRW8NjxZoWPP...,None,None
...,...,...,...,...,...,...,...
268,3KH8lL0YigCOb8ndVndxgR,James Baby,Stro Elliot,[],https://open.spotify.com/track/3KH8lL0YigCOb8n...,None,None
269,668XfwYfRJ6R1QtV6bELXu,Boom,Ta-ku,[],https://open.spotify.com/track/668XfwYfRJ6R1Qt...,None,None
270,2iAvOMJknYUX8W06ozwoJi,1969,El Ten Eleven,[],https://open.spotify.com/track/2iAvOMJknYUX8W0...,None,None
271,33KvuePYdzsx7ZqoTj3rHU,Rock the Bells,Boys Noize,[],https://open.spotify.com/track/33KvuePYdzsx7Zq...,None,None


In [33]:
# Delete duplicate songs
df_songs = df_songs.drop_duplicates(subset=['spotify_id'], keep='first')

## Get lyrics
Used lyrics genius API to get lyrics

In [24]:
import lyricsgenius
genius_token = 'YOUR-TOKEN'
genius = lyricsgenius.Genius(genius_token, skip_non_songs=True, 
                             excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True) 

In [25]:
for ind in df_songs.index:
    song = genius.search_song(df_songs['title'][ind], df_songs['artist'][ind])
    
    if song is not(None):
        # Add information from genius to the songs database
        df_songs['song_art_image_url'][ind] = song.to_dict()['song_art_image_url']
        df_songs['lyrics'][ind] = song.to_dict()['lyrics']

Searching for "Roses" by SAINt JHN...
Done.
Searching for "My Affection " by Summer Walker...


<ipython-input-25-298ff32fc5dc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_songs['song_art_image_url'][ind] = song.to_dict()['song_art_image_url']
<ipython-input-25-298ff32fc5dc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_songs['lyrics'][ind] = song.to_dict()['lyrics']


Done.
Searching for "Mood Swings" by A Boogie Wit da Hoodie...
Done.
Searching for "Chicago Freestyle " by Drake...
Done.
Searching for "Tadow" by Masego...
Done.
Searching for "The West" by Kai Ca$h...
Done.
Searching for "No Idea" by Don Toliver...
Done.
Searching for "Cardigan" by Don Toliver...
Done.
Searching for "We Paid " by Lil Baby...
Done.
Searching for "Yes Indeed" by Lil Baby...
Done.
Searching for "Lemonade" by Internet Money...
Done.
Searching for "Drinking and Driving" by Various Artists...
Done.
Searching for "No Good" by dvsn...
Done.
Searching for "John Redcorn" by SiR...
Done.
Searching for "Nuh Time / Tek Time" by dvsn...
Done.
Searching for "Lost" by Maroon 5...
Done.
Searching for "fantasize" by ericdoa...
Done.
Searching for "What You Say" by Cold War Kids...
Done.
Searching for "Live to Survive" by MØ...
Specified song does not contain lyrics. Rejecting.
Searching for "Want" by Q...
Done.
Searching for "Mon Cheri" by Sofi Tukker...
Done.
Searching for "Rookie Of

Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)

In [54]:
no_lyrics = df_songs['lyrics'].isna().sum()
tot_songs = df_songs.shape[0]
perc_songs = (((tot_songs - no_lyrics) / tot_songs) * 100).round(2)
print(f"Lyrics retrieved for {tot_songs - no_lyrics} out of {tot_songs} songs ({perc_songs}%)")

Lyrics retrieved for 202 out of 273 songs (73.99%)


In [26]:
df_songs['lyrics'][0]

"Roses\nI walked in the corner with the body screaming dolo\nNever sold a bag but look like Pablo in a photo\nThis gon' make 'em feel the way like Tony killed Manolo\nYou already know though, you already know though\nI walk in the corner with the money, on my finger\nShe might get it popping, I might wife her for the winter\nI already know, already know, nigga roses\nAll I need is roses\n\nTurn up baby, turn up, when I turn it on\nYou know how I get too lit when I turn it on\nCan't handle my behavior when I turn it on\nToo fast, never ask, if the life don't last\nDone been through it all\nFuck with a nigga raw, this who you wanna be\nAnd I know you won't tell nobody nothing\nAnd I know you won't tell nobody no\n\nRoses\nI might pull up flexing on these niggas like aerobics\nI might tell her girl you cute but balling\nThat shit gorgeous\nStanding on the table, Rosé, Rosé, fuck the waters\nYou know who the god is\nTurn up baby, turn up, when I turn it on\nYou know how I get too lit when 

In [58]:
# Save the dataset
df_song_new.to_csv(current_path + 'nike_run_club_songs.csv')